In [112]:
import pandas as pd
import numpy as np
import re, json, requests  
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup  


In [113]:
ctx=context() 
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config
ctx.get_tempalte_dir()

'C:\\Users\\timko\\source\\repos\\PythonBoiler\\CodeGenerators\\snippets\\'

In [188]:
def generate_code_from_db(qgroup):
    df=sql_todf("""
    SELECT 
idText as identifier_text 
,CASE WHEN idText='' THEN REPLACE(ID,'-','_')  ELSE REPLACE(idText,'.','_') END AS [{idt}]
,ID as [id]
,Q_TypeCode CTRLCODE
,FK_QuestionType
,PK_Question as [{PK_Question}]
,sortpos
,QTEXT as [{QuestionText}] 
    """+ f"FROM vwQuestions WHERE PK_QuestionGroup={qgroup}", connstr) 
    df['{idt}']=df['{idt}'].apply(lambda s: re.sub('_$','',s))
    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    return df, code
df,code=generate_code_from_db(4018)
#df[:10]
with open('C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\FismaForms\\2023\\2023_Q1_CIO_5.aspx', 'r', encoding='utf-16') as f:
    s=f.read() 
q=re.findall('<!--questions-->.+<!--\/questions-->',s,re.DOTALL)[0]
s=s.replace(q,f'<!--questions-->\n{code}\n<!--/questions-->')
with open(f"{ctx.get_dest()}script.aspx", 'w', encoding='utf-16') as f:
    f.write(s)


{'PK_QuestionTypeId': 1, 'Code': 'YN', 'description': 'Yes / No'}

In [175]:
options = Options()
options.add_argument("--start-minimized")  
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install() ,chrome_options=options)  



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST driver version for 107.0.5304
Driver [C:\Users\timko\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache
<ipython-input-175-4332031c7886>:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install() ,chrome_options=options)


In [176]:
driver.get(f"https://dayman.cyber-balance.com/jira/login.jsp")  
driver.find_element(By.XPATH, '//input[contains(@id, "user")]').send_keys(ctx.config['user'])
driver.find_element(By.XPATH, '//input[contains(@id, "pass")]').send_keys(ctx.config['pass']) 
driver.find_element(By.XPATH, '//input[contains(@id, "submit")]').click() 

In [ ]:
def parse_metrics(tnum, pk, met_parse='(\d{1,2}\.\d{1,2}\.?[\.\d\w]?)\s+[A-Z]'):
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")  
    soup = BeautifulSoup(driver.page_source)
    le = soup.select('td.confluenceTd, p')
    lod=[]
    for i,e in enumerate(le):
        m=re.search(met_parse,str(e.text))    
        if(m):  
            lod.append({'identifier_text':m.groups(1)[0]
                ,'QuestionText':e.text.replace(m.groups(1)[0],'')
                ,'{PK_Question}':len(lod)+1+pk
                ,'FK_QuestionType':get_question_type(e.text, ctx)['PK_QuestionTypeId']
                ,'sortpos':i
            }) 
    df=pd.DataFrame(lod)
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read()
        for i,r in df.iterrows():
            for c in df.columns: 
                df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
        sql='\n,'.join(df['sql'])
    return df, sql
#df,sql = parse_metrics('9302', 36421,  met_parse='(10\.\d{1,2}\.?[\.\d\w]?)\s+[A-Z]')
#df_to_insert(df)
#print(sql)

In [ ]:

def parse_grid(tnum, pk, QCol='Question', PivotCols=4): 
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")   
    ldf = pd.read_html(driver.page_source, match="\d\.\d\.\d", header=1)
    df=ldf[0].iloc[:,:].rename(columns={QCol: "QuestionText"})
 
    df['identifier_text'] = df['QuestionText'].apply(lambda s: re.findall('^[\d\.\w]{1,7}',s)[0])
    stypes = {k: chr(v+97) for v, k in enumerate(list(df.columns[1:4]))} 
    print(stypes)
    df=df.loc[:,['QuestionText','identifier_text']] 
    df['CTRLCODE']='LABEL'
    lod=df.to_dict(orient='records')
    l=[]
    for i,r in enumerate(lod):
        l.append(lod[i])
        for k in stypes.keys():
            id=lod[i]['identifier_text']+'.'+stypes[k]
            l.append({
                'identifier_text':id
                , 'QuestionText': id+' '+k
                , 'CTRLCODE':'CNT'
                })
    df=pd.DataFrame(l)
    df['{PK_Question}'] = range(pk,len(df)+pk)
    df['sortpos'] = range(1,len(df)+1) 
    df['FK_QuestionType'] = df['CTRLCODE'].apply(lambda s: 2 if s=='CNT' else 18)
    df['QuestionText']=df['QuestionText'].apply(lambda s: re.sub('^[\d\.\w]{1,7}','',s, count=1).strip())
    df['{QuestionText}']=df['QuestionText']
    df['{idt}'] = df['identifier_text'].str.replace('.','_')
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read() 
    for i,r in df.iterrows():
        for c in df.columns: 
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)

    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    
    return df,sql,code

# df,sql,code=parse_grid('9295',37400, QCol='Type of Test')
# print(sql)

In [80]:
with open('C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\FismaForms\\2023\\2023_Q1_CIO_2.aspx', 'r', encoding='utf-16') as f:
    s=f.read()
    s=re.sub('uc\d','uc',s)
q=re.findall('<!--questions-->.+<!--\/questions-->',s,re.DOTALL)[0]
s=s.replace(q,code)
with open('script.aspx', 'w', encoding='utf-16') as f:
    f.write(s)

In [139]:
def df_to_insert(dff):
    df=dff
    df=df.astype(str)
    df.reset_index(inplace=True)    
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read()   
    for i,r in df.iterrows():
        for c in df.columns:   
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),str(df.loc[i,'sql']))  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)
    print(sql)    

In [ ]:
df=pd.DataFrame(range(1,2))
df["identifier_text"]=df[0].apply(lambda s : '3.3.'+str(s))
df['CTRLCODE']='LABEL'
df['m']=[['a Tier EL0 Not Effective','b Tier EL1 Basic','c Tier EL2 Intermediate','d Tier EL3 Advanced'] for i in range(len(df))]
lbls=df
df=df.explode('m')
df=pd.concat([lbls,df]) 
df['CTRLCODE']=df['m'].apply(lambda s: 2 if type(s)==str else 18)
df.sort_values([0,'CTRLCODE'], ascending=[True,False], inplace=True) 
df['m']=df['m'].apply(lambda s: s if type(s)!=list else '') 
df['QuestionText']=  df['m'].apply(lambda s: s[2:])
df["identifier_text"]=df["identifier_text"]+ ''+ df['m'].apply(lambda s: s.split(' ')[0])
df['{PK_Question}'] = range(37250,len(df)+37250)
df['sortpos'] = range(50,len(df)+50) 
df['FK_QuestionType'] = df['CTRLCODE']
df['CTRLCODE'] = df['FK_QuestionType'].apply(lambda s: 'CNT' if s==2 else 'LABEL')  
df['{idt}'] = df['identifier_text'].str.replace('.','_')
df.drop(columns=[0,'m'], inplace=True)
df
df_to_insert(df)


In [ ]:
df=sql_todf("SELECT QTEXT, IdText, Q_TypeCode FROM vwQuestions WHERE PK_FORM='2023-A-HVA' ORDER BY GroupName asc,  SortPos asc", connstr)
df

,identifier_text,{idt},id,CTRLCODE,FK_QuestionType,{PK_Question},sortpos,{QuestionText}
0,2.1,2_1,5-1-2.1,LABEL,18,37000,1,How many systems (from 1.1.1 and 1.1.2) encryp...
1,2.1.a,2_1_a,5-2-2.1.a,CNT,2,37001,2,Number of FISMA Low Systems
2,2.1.b,2_1_b,5-3-2.1.b,CNT,2,37002,3,Number of FISMA Moderate Systems
3,2.1.c,2_1_c,5-4-2.1.c,CNT,2,37003,4,Number of FISMA High Systems
4,2.2,2_2,5-5-2.2,LABEL,18,37004,5,How many systems (from 1.1.1 and 1.1.2) will o...
5,2.2.a,2_2_a,5-6-2.2.a,CNT,2,37005,6,Number of FISMA Low Systems


In [ ]:
p=questionnaire_parser(ctx)
df=p.parse()
df.to_html('parsed.html')
df.to_csv('parsed.csv')

In [ ]:
input="""
What is the approximate time in years since deployment of this HVA?
 
""".split('\n')

pr = picklist_recommender(connstr)
r=pr.recommend(input, threshhold=.8, usecosine_sim=False)
r

In [ ]:
df=pd.read_csv(r'C:\Users\timko\Downloads\Agency Missing CVEs.csv').loc[:, ['cve_id', 'acronym','cdm_findings']]
df=pd.merge(df,cl,how='left', left_on='acronym', right_on='acronym')
df.loc[df['PK_component'].isnull()]

In [ ]:
def to_sql(t):
    cve_id,acronym,cdm_findings=t
    s=f"""
    INSERT INTO @T(PK_OrgSubmission, PK_CISA_CVE, CDMCountSubmitted, CDMDateSubmitted, SubmissionCount,UserId)
    VALUES ( 
         (SELECT MAX(PK_OrgSubmission) FROM vwOrgSubToComponent WHERE Acronym='{acronym}' AND PK_ReportingCycle=108 AND OrgSub_Description='ACTIVE' AND OrgSub_Status='SUBO')
        , '{cve_id}', {cdm_findings} , '11-14-2022' , {cdm_findings} , 2815 )
    """ 
    return s
df=pd.read_csv(r'C:\Users\timko\Downloads\Agency Missing CVEs.csv').loc[:, ['cve_id', 'acronym','cdm_findings']]
df=pd.merge(df,cl,how='left', left_on='acronym', right_on='acronym')
df=df.sort_values(by='acronym')
df['sql']=[to_sql(i) for i in zip( df['cve_id'], df['acronym'], df['cdm_findings'] )]

s=''.join([i for i in df['sql']])
with open('script.sql', 'w') as f:
    f.write(s)
#print(s)